In [21]:
from matplotlib import pyplot as plt
import sys
from comicimagelib import *

In [22]:
mypath='image_input/'

In [23]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [24]:
fname = onlyfiles[0]

In [26]:
img = cv2.imread(mypath+onlyfiles[0])
print('image size:', img.shape)
# check if image is black/white or color
if is_grey_scale(img):
    exit("black and white images not yet supported...")

res = image_color_clust(img, 4, True)
bgcolor = get_background_color(res)
print("background color:", bgcolor)
res[np.where((res == bgcolor).all(axis=2))] = [255,255,255]
# cv2.floodFill(res, None, (res.shape[1]-1, res.shape[0]-1), (255, 255, 255), loDiff=(0, 0, 0, 0), upDiff=(0, 0, 0, 0))
# cv2.floodFill(res, None, (0, 0), (255, 255, 255), loDiff=(0, 0, 0, 0), upDiff=(0, 0, 0, 0))
cv2.imwrite('image_output/' + fname + '_no_bckground.jpg', res)
gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
gray[:, (gray.shape[1] - 1)] = 255
gray[:, 0] = 255
gray[0, :] = 255
gray[gray.shape[0]-1, :] = 255
cv2.imwrite('image_output/'+ 'gray_' + fname, gray)
ret, binary = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)
cv2.imwrite('image_output/' + 'binary_' + fname, binary)
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

hierarchy = hierarchy[0] # get the actual inner list of hierarchy descriptions
font = cv2.FONT_HERSHEY_COMPLEX
area_lim = np.array(img).shape[0]*np.array(img).shape[1] // 100
mask = np.zeros(img.shape[:2], dtype="uint8") * 255

panel_dict = {}
# Get panels out of contours
for i, component in enumerate(zip(contours, hierarchy)):
    contour = component[0]
    chier = component[1]
    area = cv2.contourArea(contour)
    if area < area_lim:
        continue
    if chier[3] < 0:
        continue
    # if area >= np.array(img).shape[0]*np.array(img).shape[1] - 10000:
    #     continue
    arclength = cv2.arcLength(contour, True)
    epsilon = 0.01 * arclength
    approx = cv2.approxPolyDP(contour, epsilon, True)

    if len(approx) == 4:
        x, y, w, h = cv2.boundingRect(approx)
        panel_center = np.ceil(img.shape[1]*(y+y+h)/2 + (x+x+w)/2)
        print(panel_center)
        panel_dict[panel_center] = img[y:y+h, x:x+w].copy()
        # cv2.putText(img, "Rectangle", (x, y), font, 2, (0))
        # cv2.drawContours(mask, [contour], -1, 0, -1)
        cv2.fillPoly(img, pts=[contour], color=(255, 255, 255))

for i, key in enumerate(sorted(panel_dict.keys())):
    cv2.imwrite('image_output/' + str(i) + fname,  panel_dict[key])

cv2.imwrite('image_output/script1_residual.jpg', img)


image size: (3056, 1988, 3)
[254 253 255]
80 984
background color: [252 252 255]
3149977.0
1488499.0
1389149.0


True